In [ ]:
import torch
from diffusers import DiffusionPipeline, AutoencoderTiny

pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/sd-turbo", torch_dtype=torch.float16
)
pipe.vae = AutoencoderTiny.from_pretrained(
    "madebyollin/taesd", torch_dtype=torch.float16
)

In [53]:
pipe = pipe.to("cuda")

prompt = "slice of delicious New York-style cheesecake topped with berries, mint, chocolate crumble"
image = pipe(
    prompt,
    num_inference_steps=1,
    generator=torch.Generator("cuda").manual_seed(0x7A35D),
).images[0]
# image.save("cheesecake.png")

  0%|          | 0/1 [00:00<?, ?it/s]

In [54]:
pipe.save_pretrained("inference_optimization/models/sdturbo_taesd")

In [ ]:
from optimum.onnxruntime import ORTDiffusionPipeline

model_id = "stabilityai/sd-turbo"
pipeline = ORTDiffusionPipeline.from_pretrained(model_id, export=True)
prompt = "sailing ship in storm by Leonardo da Vinci"
image = pipeline(prompt).images[0]

In [1]:
import torch
import torch_tensorrt
from diffusers import DiffusionPipeline

# model_id = "CompVis/stable-diffusion-v1-4"
model_id = "stabilityai/sd-turbo"
device = "cuda:0"

# Instantiate Stable Diffusion Pipeline with FP16 weights
pipe = DiffusionPipeline.from_pretrained(
    # model_id, revision="fp16", torch_dtype=torch.float16
    model_id,
    torch_dtype=torch.bfloat16,
)
pipe = pipe.to(device)

backend = "torch_tensorrt"

# Optimize the UNet portion with Torch-TensorRT
pipe.unet = torch.compile(
    pipe.unet,
    backend=backend,
    options={
        "truncate_long_and_double": True,
        "enabled_precisions": {torch.bfloat16},
    },
    dynamic=False,
)

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
Unable to import quantization op. Please install modelopt library (https://github.com/NVIDIA/TensorRT-Model-Optimizer?tab=readme-ov-file#installation) to add support for compiling quantized models
TensorRT-LLM is not installed. Please install TensorRT-LLM or set TRTLLM_PLUGINS_PATH to the directory containing libnvinfer_plugin_tensorrt_llm.so to use converters for torch.distributed ops


[02/08/2025-01:52:06] [TRT] [W] Functionality provided through tensorrt.plugin module is experimental.


Loading pipeline components...: 100%|██████████| 5/5 [00:00<00:00,  8.38it/s]
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [2]:
import torch
from diffusers import DiffusionPipeline, AutoencoderTiny

pipe.vae = AutoencoderTiny.from_pretrained(
    "madebyollin/taesd", torch_dtype=torch.bfloat16
)
pipe.vae.to("cuda")
pipe.set_progress_bar_config(disable=True)
None

In [3]:
prompt = "a majestic castle in the clouds"
image = pipe(
    prompt,
    num_inference_steps=50,
).images[0]

BackendCompilerFailed: backend='torch_tensorrt' raised:
TypeError: Unsupported numpy dtype

While executing %div_1 : [num_users=2] = call_function[target=torch.ops.aten.div.Tensor](args = (%add_1, 1.0), kwargs = {_itensor_to_tensor_meta: {<tensorrt_bindings.tensorrt.ITensor object at 0x7478ad56c170>: ((2, 320, 64, 64), torch.bfloat16, False, (1310720, 4096, 64, 1), torch.contiguous_format, False, {}), <tensorrt_bindings.tensorrt.ITensor object at 0x7478ad56d1f0>: ((2, 320, 64, 64), torch.bfloat16, False, (1310720, 4096, 64, 1), torch.contiguous_format, False, {}), <tensorrt_bindings.tensorrt.ITensor object at 0x7478aeb47070>: ((320, 320, 3, 3), torch.bfloat16, True, (2880, 9, 3, 1), torch.contiguous_format, False, {}), <tensorrt_bindings.tensorrt.ITensor object at 0x7478adaccef0>: ((320,), torch.bfloat16, True, (1,), torch.contiguous_format, False, {}), <tensorrt_bindings.tensorrt.ITensor object at 0x7478ae519fb0>: ((2, 320, 64, 64), torch.bfloat16, False, (1310720, 4096, 64, 1), torch

In [6]:
import time
with torch.no_grad():
    start = time.time()
    for i in range(140):
        prompt = "a majestic castle in the clouds"
        image = pipe(
            prompt,
            num_inference_steps=1,
        ).images[0]
    full_time = time.time() - start
    print("multiple_images", full_time)
    print("multiple_images fps", 1 / (full_time / 140))

multiple_images 7.975292444229126
multiple_images fps 17.55421521894199
